In [1]:
from diffusers import UNet2DModel, DDPMScheduler
from torchvision.transforms import ToPILImage
import torch


In [2]:
image_size = 64

device = torch.device("cuda" if- torch.cuda.is_available() else "cpu")
print("Device:", device)


In [3]:
def generate_image(model, scheduler, image_size, device):
    noisy_image = torch.randn(1, 3, image_size, image_size).to(device)

    for t in reversed(range(scheduler.config.num_train_timesteps)):
        t_tensor = torch.tensor([t], device=device).long()
        with torch.no_grad():
            noise_pred = model(noisy_image, t_tensor).sample

        noisy_image = scheduler.step(
            noise_pred, t_tensor, noisy_image
            ).prev_sample

    image = (noisy_image.clamp(-1, 1) + 1) / 2  # Scale the image to [0, 1]
    image = image.cpu().squeeze()

    return image


In [4]:
scheduler = DDPMScheduler(
    num_train_timesteps=1000,
    beta_start=0.0001,
    beta_end=0.02,
    beta_schedule="linear"
)

model = UNet2DModel.from_pretrained("model_paths/all_data_1")
model.to(device)
model.eval()

generated_image = generate_image(model, scheduler, image_size, device)

to_pil = ToPILImage()
pil_image = to_pil(generated_image)
pil_image.show()


Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
